##### Questions about LLM

In [4]:
# Generate per-point, one-page PDFs with expanded answers (RU & EN).
import matplotlib.pyplot as plt
import textwrap
from matplotlib.backends.backend_pdf import PdfPages

# ---------- Helpers ----------
def draw_page(title, sections, out_pdf, lang='ru'):
    """
    sections: list of tuples (heading, text) where text may include '\n' and bullets prefixed with '• '
    Creates a single page appended to out_pdf (PdfPages).
    """
    # A4 portrait
    fig_w, fig_h = 8.27, 11.69
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.axis('off')
    
    # Title
    ax.text(0.5, 0.96, title, ha='center', va='top', fontsize=14, fontweight='bold', wrap=True)
    
    y = 0.92
    line_space = 0.018  # vertical spacing per wrapped line (axes coords)
    wrap_cols = 98 if lang == 'ru' else 100
    
    for heading, text in sections:
        if y < 0.06:  # new column/panel not supported; ensure we don't overflow
            ax.text(0.5, 0.03, ("(продолжение не влезло)" if lang=='ru' else "(content truncated)"), 
                    ha='center', va='bottom', fontsize=8)
            break
        
        # Heading
        ax.text(0.05, y, heading, ha='left', va='top', fontsize=10, fontweight='bold')
        y -= line_space * 1.2
        
        # Body text -> wrap and render line by line
        for paragraph in text.split("\n"):
            if not paragraph.strip():
                y -= line_space * 0.5
                continue
            # wrap lines; keep bullets if present
            wrapped = textwrap.wrap(paragraph, width=wrap_cols, break_long_words=False, break_on_hyphens=False)
            if not wrapped:
                wrapped = [""]
            for line in wrapped:
                ax.text(0.05, y, line, ha='left', va='top', fontsize=9)
                y -= line_space
        y -= line_space * 0.6
    
    # Footer
    ax.text(0.5, 0.015, "Распечатайте как PDF / Print to PDF" if lang=='ru' else "Print to PDF",
            ha='center', va='bottom', fontsize=8, color="#444444")
    
    out_pdf.savefig(fig, dpi=300, bbox_inches='tight')
    plt.close(fig)

# ---------- Content (RU) ----------
ru_items = [
    (
        "1) MQA / GQA и KV-кэш",
        [
            ("Вопрос", "Зачем MQA/GQA и как это влияет на размер/скорость KV-кэша?"),
            ("Что должен затронуть профи",
             "• MQA: один общий K/V для всех Q-голов; GQA: несколько групп Q делят меньшее число K/V-голов.\n"
             "• Экономия памяти и HBM-трафика ~ пропорционально отношению n_heads / n_kv_heads.\n"
             "• Влияние на качество: обычно минимальное, особенно в чат-задачах; иногда падают длинные зависимости."),
            ("Развернутый ответ",
             "В multi-head attention каждый из h голов имеет собственные K и V, поэтому KV-кэш растёт как O(h·L·d_head·layers). "
             "MQA задаёт n_kv_heads=1 (или сильно меньше h), GQA — n_kv_heads между 1 и h (например, 8 при 64 Q-головах). "
             "В итоге KV-кэш и объём чтений из HBM уменьшаются почти в h/n_kv_heads раз, что особенно важно на длинных контекстах."
             "\n\nПрактически это снижает латентность и повышает пропускную способность при decode. Цена — небольшая потеря качества "
             "на задачах с тонкими межголовными корреляциями; для большинства прикладных LLM компромисс оправдан."),
            ("Red flags",
             "• Путает multi-query с multi-head.\n• Не упоминает экономию KV/памяти и HBM-трафика.\n• Утверждает, что «качество не меняется никогда».")
        ]
    ),
    (
        "2) PagedAttention (vLLM)",
        [
            ("Вопрос","Что именно «пейджится» и почему это быстрее?"),
            ("Что должен затронуть профи",
             "• KV-кэш хранится в фиксированных блоках (страницах) одинакового размера.\n"
             "• O(1) перераспределение кэша без копирований, меньше фрагментации.\n"
             "• Поддержка continuous batching и быстрая утилизация префиксов."),
            ("Развернутый ответ",
             "Вместо линейного массива под каждый запрос, vLLM хранит KV-кэш как набор страниц. "
             "Когда один запрос заканчивается, его страницы переиспользуются для другого без больших копирований. "
             "Это резко снижает накладные расходы при динамическом потоке разной длины, упрощает partage KV-префиксов и "
             "повышает загрузку GPU. Алгоритм внимания не меняется — меняется менеджмент памяти."),
            ("Red flags",
             "• Называет это «новым видом внимания».\n• Не упоминает блоковую разметку и проблему фрагментации.")
        ]
    ),
    (
        "3) FlashAttention-3",
        [
            ("Вопрос","Чем FA-3 лучше v2 и где даёт выигрыш?"),
            ("Что должен затронуть профи",
             "• Более IO-эффективная тилация/конвейеризация.\n• FP8/Tensor Cores (Hopper/Blackwell).\n• Меньше HBM IO → ~1.5–2× в ряде режимов."),
            ("Развернутый ответ",
             "FA-семейство минимизирует лишние чтения/записи при вычислении softmax(QKᵀ)V, укладывая данные в SRAM и обрабатывая их «плитками». "
             "Версия 3 оптимизирует пайплайны под новые архитектуры (Tensor Cores, FP8), снижает трафик к HBM и ускоряет train/infer. "
             "На малых head_dim/seq_len выигрыш заметно меньше — величина зависит от формы тензоров."),
            ("Red flags",
             "• Путает с обычным scaled dot-product.\n• Не говорит про IO-aware дизайн и привязку к архитектуре GPU.")
        ]
    ),
    (
        "4) Speculative decoding",
        [
            ("Вопрос","Когда выгоден и где узкое место?"),
            ("Что должен затронуть профи",
             "• Быстрый «драфтер» предлагает несколько токенов, большая модель их верифицирует.\n"
             "• Важно: acceptance rate, стоимость верификации, синхронизация KV.\n• Падение профита при длинных зависимостях."),
            ("Развернутый ответ",
             "Если драфтер 2–4× быстрее и принимает ≥40–60% предложений, итоговый TPS растёт. "
             "Но когда приём низок, постоянные откаты и проверка «съедают» выигрыш. "
             "Критично грамотно кэшировать/сливать KV, чтобы не дублировать работу. Хорошо работает на коротких/средних зависимостях."),
            ("Red flags",
             "• «Всегда быстрее, ставь и не думай». \n• Игнорирует acceptance и накладные расходы верификации.")
        ]
    ),
    (
        "5) Память KV: формула и пример",
        [
            ("Вопрос","Как оценить память KV и почему она растёт с L?"),
            ("Что должен затронуть профи",
             "• Формула: bytes ≈ batch·L·layers·n_kv_heads·head_dim·2(K,V)·bytes_per_el.\n"
             "• Пример на типичных параметрах ≈ десятки ГБ при больших L."),
            ("Развернутый ответ",
             "KV-кэш хранит K и V для каждого токена и слоя. Поэтому рост ~O(L). "
             "Пример: b=1, L=8192, layers=80, n_kv_heads=32, head_dim=128, fp16 (2 B) → ~10.7 ГБ. "
             "При батче 4 это уже ~43 ГБ. Именно из-за KV-кэша длинные контексты дороги в памяти и пропускной способности."),
            ("Red flags",
             "• Считает только V или забывает K.\n• Путает dtype и байты на элемент.")
        ]
    ),
    (
        "6) Prefill vs Decode",
        [
            ("Вопрос","Почему prefill обычно дороже и как это влияет на батчинг?"),
            ("Что должен затронуть профи",
             "• Prefill ~O(L²) по вниманию, decode ~O(L) на шаг.\n• Prefill выгодно крупно батчить/чанкать; decode — latency-критичен."),
            ("Развернутый ответ",
             "Prefill обрабатывает весь промпт сразу, поэтому матмулы и внимание масштабируются квадратично по длине. "
             "Decode добавляет по одному токену, O(L) на шаг, и упирается в задержки. "
             "Практика: крупные батчи/чанкинг на prefill, на decode — continuous batching и оптимальный размер групп."),
            ("Red flags",
             "• Утверждает, что decode всегда тяжелее.\n• Игнорирует разницу в батчинге стадий.")
        ]
    ),
    (
        "7) Continuous batching и prefix-sharing",
        [
            ("Вопрос","Что это и чем помогает?"),
            ("Что должен затронуть профи",
             "• Приём запросов «на лету», объединение по стадиям.\n• Sharing KV-префикса для одинаковых начал.\n• Рост утилизации и падение latency."),
            ("Развернутый ответ",
             "Система добавляет новые запросы не дожидаясь завершения старых, совмещая их по стадиям (prefill/decode). "
             "Если у запросов общий пролог (системный промпт, инструкция), их KV-страницы переиспользуются. "
             "Результат — меньше матмулов и лучшее заполнение GPU."),
            ("Red flags",
             "• Не различает prefill/decode.\n• Не знает про reuse KV/префиксы.")
        ]
    ),
    (
        "8) RoPE-скейлинг (NTK, YaRN)",
        [
            ("Вопрос","Когда применять и какой риск?"),
            ("Что должен затронуть профи",
             "• Удлинение контекста без полного ретрейна.\n• Риск деградации качества, особенно «в середине» и на очень длинных L.\n• Лучше дообучать на длинных примерах."),
            ("Развернутый ответ",
             "RoPE задаёт позиционные фазы. Скейлинг корректирует частоты, чтобы модель экстраполировала дальше базового контекста. "
             "Без адаптации растут артефакты: ухудшение точности, внезапные «провалы» на средних позициях. "
             "Лучше сочетать с do-train/LoRA на длинных образцах и аккуратно подбирать коэффициенты."),
            ("Red flags",
             "• «Бесплатно и без потерь». \n• Не знает про NTK-aware/YaRN и побочные эффекты.")
        ]
    ),
    (
        "9) Sliding-window attention (SWA)",
        [
            ("Вопрос","Плюсы/минусы относительно полного self-attention?"),
            ("Что должен затронуть профи",
             "• Сложность ~O(L·T) вместо O(L²), но теряются дальние зависимости.\n• Компенсаторы: глобальные токены/суммаризация/внешняя память."),
            ("Развернутый ответ",
             "SWA ограничивает внимание окном последних T токенов. Это даёт линейную по L сложность и большой выигрыш на длинных контекстах, "
             "но ухудшает понимание далеких фактов. Практичные дизайны добавляют немного «глобальных» позиций или используют RAG/память."),
            ("Red flags",
             "• «Ничего не теряем».\n• Не предлагает компенсирующие механизмы.")
        ]
    ),
    (
        "10) SFT / RLHF / DPO-семейство",
        [
            ("Вопрос","Когда что выбирать для alignment?"),
            ("Что должен затронуть профи",
             "• SFT — базовый стиль/послушание.\n• DPO/ORPO — предпочтения без reward-модели.\n• RLHF — тонкая настройка, дороже и менее стабильна."),
            ("Развернутый ответ",
             "SFT формирует следование инструкциям. DPO/ORPO учат выбирать лучшую из пары ответов, минуя явный RM, дешевле и стабильнее. "
             "RLHF требует RM и PPO-подобный шаг, но умеет «вытащить» поведение в сложных зонах (безопасность, тон). "
             "Комбинации: SFT → DPO → лёгкий RLHF на сложных кейсах."),
            ("Red flags",
             "• Считает DPO = RLHF.\n• «Достаточно одного SFT всегда».")
        ]
    ),
    (
        "11) Данные: отбор и дедуп",
        [
            ("Вопрос","Что критично в данных претрейна/инструкта?"),
            ("Что должен затронуть профи",
             "• Дедуп (MinHash/SimHash), фильтры качества/токсичности/PII.\n• Баланс доменов и sampling.\n• Метрики: PPL, бенчмарки, регрессионные тесты."),
            ("Развернутый ответ",
             "Качество данных доминирует. Удаляем дубликаты и спам, выравниваем доли кода/языков/домена, применяем quality-классификаторы. "
             "Делим на чистые валидационные сплиты без утечек. Следим за PPL/accuracy на эталонных наборах и регрессиями при добавлении новых корпусов."),
            ("Red flags",
             "• «Больше данных = всегда лучше».\n• Нет дедупа и контроля качества.")
        ]
    ),
    (
        "12) Токенизация: BPE vs Unigram",
        [
            ("Вопрос","Как влияет выбор на код/числа?"),
            ("Что должен затронуть профи",
             "• Unigram гибче в сегментации; BPE — фикс-мерджи.\n• Для кода/чисел нужны спец-разделители/токены."),
            ("Развернутый ответ",
             "Byte-level BPE надёжна, но для чисел без явных разделителей длина растёт. Unigram моделирует вероятностное разбиение субтокенов, "
             "даёт компактнее представление в некоторых доменах. Практика — кастомные правила для чисел/символов кода и контроль длины последовательностей."),
            ("Red flags",
             "• «Разницы нет».\n• Игнорирует особенность кода/чисел.")
        ]
    ),
    (
        "13) Оптимизаторы и точности",
        [
            ("Вопрос","Дефолтные связки и что меняется на Hopper/Blackwell?"),
            ("Что должен затронуть профи",
             "• AdamW + bfloat16, warmup→cosine, grad-clip, fused-ядра.\n• На новых GPU — FP8 с калибровкой скейлов."),
            ("Развернутый ответ",
             "bfloat16 устойчивее FP16 при той же памяти. AdamW остаётся стандартом, но важны fused-операции и профилирование. "
             "На Hopper/Blackwell FP8 даёт throughput, но требует калибровки скейлов/порогов и мониторинга стабильности. "
             "ZeRO/gradient-checkpointing — для экономии памяти; аккуратно с интенсификацией коммуникаций."),
            ("Red flags",
             "• «FP16 везде достаточно».\n• Не учитывает bfloat16/FP8 и fused-ядра.")
        ]
    ),
    (
        "14) Параллелизм и железо",
        [
            ("Вопрос","TP/PP/DP; MIG/NVLink/SR-IOV — как применять?"),
            ("Что должен затронуть профи",
             "• TP — шард матриц; PP — деление слоёв; DP — копии.\n• NVLink/NVSwitch критичны для TP/PP; MIG — QoS/изоляция; SR-IOV — виртуализация."),
            ("Развернутый ответ",
             "При TP матрицы разбиваются по устройствам, при PP — модель делится на этапы по слоям, DP — обычное распределение батчей. "
             "Большой TP/PP требует быстрой межсвязи — NVLink/NSwitch; PCIe часто узкое место. MIG делит GPU на изолированные слайсы; "
             "SR-IOV — раздаёт виртуальные функции, полезно в мульти-тенант окружении."),
            ("Red flags",
             "• Считает PCIe «норм» для большого TP.\n• Не понимает ограничения межсоединений.")
        ]
    ),
    (
        "15) «Модель всё помнит» — проверка",
        [
            ("Вопрос","Как валидировать тезис о «памяти навсегда»?"),
            ("Что должен затронуть профи",
             "• Развести контекстное окно (KV) и долговременную память (RAG/база).\n• Мерить latency/стоимость и метрики ретривала (R@k, MRR)."),
            ("Развернутый ответ",
             "Веса — не долговременная адресуемая память. Для реальной памяти нужна внешняя БД/векторное хранилище и retrieval. "
             "Контекст ограничен KV-кэшем и дорог в обслуживании. Архитектуры прод-систем: «инференс-движок + кеш KV + индекс (RAG) + ранкер» "
             "с политиками вытеснения и безопасностью. Обещания «помнить всё внутри» — маркетинг."),
            ("Red flags",
             "• Путает RAG с «бесконечным контекстом».\n• Не считает стоимость/латентность хранения всего в окне.")
        ]
    ),
]

# ---------- Content (EN) ----------
en_items = [
    (
        "1) MQA / GQA & the KV cache",
        [
            ("Question","Why MQA/GQA and how do they affect KV memory and speed?"),
            ("What a pro should cover",
             "• MQA: one shared K/V for all Q heads; GQA: a few K/V heads shared by groups of Q heads.\n"
             "• Memory/HBM traffic drops roughly by n_heads / n_kv_heads.\n• Quality impact usually small; long-range deps may suffer a bit."),
            ("Expanded answer",
             "In standard MHA each head keeps its own K and V, so the KV cache scales as O(h·L·d_head·layers). "
             "MQA sets n_kv_heads=1 (or much smaller than h); GQA picks a handful (e.g., 8 for 64 Q heads). "
             "This shrinks KV memory and memory traffic proportionally, crucial for long contexts where decode becomes memory-bound."),
            ("Red flags",
             "• Confuses multi-query with multi-head.\n• Never mentions KV/HBM traffic angle.\n• Claims quality never changes.")
        ]
    ),
    # (For brevity, we can mirror the RU content with English phrasing; creating all 15.)
]

# Fill EN by mirroring RU phrases succinctly
en_map = [
    ("2) PagedAttention (vLLM)","What is paged and why faster?",
     "• KV cache stored in fixed pages.\n• O(1) reallocation, less fragmentation.\n• Enables continuous batching and fast prefix reuse.",
     "Instead of linear buffers per request, vLLM stores KV in equal pages. When a request finishes, its pages are reused with almost no copying, "
     "reducing overhead with variable-length traffic and boosting GPU utilization. The attention math stays the same—only memory management changes.",
     "• Calls it “a new attention algorithm”.\n• Omits paging/fragmentation detail."
    ),
    ("3) FlashAttention-3","How is FA-3 better than v2 and where?",
     "• More IO-aware tiling/pipelines.\n• FP8/Tensor Cores (Hopper/Blackwell).\n• Less HBM IO → ~1.5–2× on some shapes.",
     "FA computes softmax(QKᵀ)V with IO-aware tiling to keep data in SRAM. v3 optimizes pipelines for new GPUs and FP8, lowering HBM traffic and speeding train/infer. "
     "Gains shrink for tiny head_dim/seq_len; speedup is shape-dependent.",
     "• Confuses with plain SDPA.\n• Ignores IO-aware design and GPU arch specifics."
    ),
    ("4) Speculative decoding","When beneficial and what’s the bottleneck?",
     "• Fast drafter proposes tokens; verifier checks.\n• Acceptance rate and verification overhead dominate.\n• Weak at very long dependencies.",
     "If the drafter is 2–4× faster")]


In [ ]:
!pip install matplotlib, textwrap